Imports

In [95]:
import json
from gensim import corpora, models, similarities
from nltk.tokenize import word_tokenize
import nltk
import os
import shutil
import numpy as np
import logging


Configuração do Logger

In [96]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Carregando .jsons

In [97]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Download de dependências do NLTK
   

In [98]:
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

Indexar o Corpus com Vetorial (TF-IDF)

In [99]:
logging.info("Construindo o modelo Vetorial (TF-IDF)...")
doc_ids = list(map_documentos.keys())
doc_texts = list(map_documentos.values())
texts = [word_tokenize(text.lower()) for text in doc_texts]
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]
tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

# --- CORREÇÃO DE MEMORYERROR ---
# A classe 'MatrixSimilarity' tenta carregar toda a matriz na RAM.
# Para corpora grandes, usamos 'Similarity', que salva o índice em arquivos
# temporários no disco, consumindo muito menos memória.
temp_folder = './temp_similarity_index'
# Garante que a pasta temporária esteja limpa antes de usar
if os.path.exists(temp_folder):
    shutil.rmtree(temp_folder)
os.makedirs(temp_folder)
output_prefix = os.path.join(temp_folder, 'shard')
# A linha abaixo substitui 'similarities.MatrixSimilarity'
index = similarities.Similarity(output_prefix, tfidf_corpus, num_features=len(dictionary))
logging.info("Modelo construído com sucesso!")

2025-07-22 05:56:17 - INFO - Construindo o modelo Vetorial (TF-IDF)...
2025-07-22 05:56:24 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2025-07-22 05:56:24 - INFO - adding document #10000 to Dictionary<8953 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 05:56:24 - INFO - adding document #20000 to Dictionary<12182 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 05:56:25 - INFO - adding document #30000 to Dictionary<16741 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 05:56:25 - INFO - adding document #40000 to Dictionary<19925 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 05:56:25 - INFO - adding document #50000 to Dictionary<22221 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 05:56:25 - INFO - adding document #60000 to Dictionary<25505 unique tok

Função que retorna Top N documentos de uma consulta Vetorial

In [100]:
def obter_top_n_vetorial(consulta_tokenizada, n=50):
    # 1. Converte a consulta para o espaço vetorial TF-IDF
    query_bow = dictionary.doc2bow(consulta_tokenizada)
    query_tfidf = tfidf_model[query_bow]

    # 2. Calcula os scores (similaridades) para cada documento no corpus
    doc_scores = index[query_tfidf]

    # 3. Pega os índices dos scores em ordem decrescente de pontuação
    top_n_indices = np.argsort(doc_scores)[::-1]

    # 4. Retorna apenas os 'n' primeiros IDs da lista ordenada
    return [int(doc_ids[idx]) for idx in top_n_indices[:n]]

Executar consulta e gerar os resultados top 50

In [101]:
logging.info("Executando consulta de exemplo...")
# Escolha uma consulta para testar
id_consulta_exemplo = "182" # Usando a primeira consulta como exemplo
texto_consulta_exemplo = map_consultas[id_consulta_exemplo]

print(f"Texto da consulta (ID: {id_consulta_exemplo}): '{texto_consulta_exemplo}'")

# Tokeniza a consulta
consulta_tokenizada = texto_consulta_exemplo.lower().split()

# Top 50 documentos mais relevantes para a consulta
resultados_vetorial = obter_top_n_vetorial(consulta_tokenizada, n=50)

logging.info(f"Encontrados {len(resultados_vetorial)} resultados.")

2025-07-22 05:56:40 - INFO - Executando consulta de exemplo...
2025-07-22 05:56:40 - INFO - creating sparse index
2025-07-22 05:56:40 - INFO - creating sparse matrix from corpus
2025-07-22 05:56:40 - INFO - PROGRESS: at document #0/31187
2025-07-22 05:56:40 - INFO - PROGRESS: at document #10000/31187


Texto da consulta (ID: 182): 'mutations sonic hedgehog genes affect developmental disorders'


2025-07-22 05:56:40 - INFO - PROGRESS: at document #20000/31187
2025-07-22 05:56:40 - INFO - PROGRESS: at document #30000/31187
2025-07-22 05:56:40 - INFO - created <31187x44922 sparse matrix of type '<class 'numpy.float32'>'
	with 256208 stored elements in Compressed Sparse Row format>
2025-07-22 05:56:40 - INFO - creating sparse shard #4
2025-07-22 05:56:40 - INFO - saving index shard to ./temp_similarity_index\shard.4
2025-07-22 05:56:40 - INFO - SparseMatrixSimilarity lifecycle event {'fname_or_handle': './temp_similarity_index\\shard.4', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-07-22T05:56:40.930677', 'gensim': '4.3.3', 'python': '3.12.8 | Intel Corporation | (main, Apr  2 2025, 09:07:08) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'saving'}
2025-07-22 05:56:40 - INFO - saved ./temp_similarity_index\shard.4
2025-07-22 05:56:40 - INFO - loading SparseMatrixSimilarity object from ./temp_similarity_index

Salvando resultados_vetorial como .json

In [102]:
nome_arquivo_saida = 'resultados_vetorial.json'
logging.info(f"Salvando resultados em '{nome_arquivo_saida}'...")

try:
    with open(nome_arquivo_saida, 'w', encoding='utf-8') as f:
        json.dump(
            resultados_vetorial,
            f,
            indent=4,
            ensure_ascii=False
        )
    logging.info("Arquivo de resultados do modelo Vetorial salvo com sucesso!")
except Exception as e:
    logging.error(f"ERRO ao salvar o arquivo JSON: {e}")
finally:
    # Limpa a pasta temporária do índice após o uso
    logging.info("Limpando arquivos temporários do índice...")
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
        print("Limpeza concluída.")

2025-07-22 05:56:41 - INFO - Salvando resultados em 'resultados_vetorial.json'...
2025-07-22 05:56:41 - INFO - Arquivo de resultados do modelo Vetorial salvo com sucesso!
2025-07-22 05:56:41 - INFO - Limpando arquivos temporários do índice...


Limpeza concluída.
